In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
df = pd.read_csv("weatherData.csv")
df = df.drop(['Loud Cover'], axis=1) #.sort_values(by="formatted_date").drop(["formatted_date"], axis=1)
df.head()

,formatted_date,summary,precip_type,temperature,apparent_temperature,humidity,wind_speed,wind_bearing,visibility,pressure,daily_summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251,15.8263,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259,15.8263,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204,14.9569,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269,15.8263,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259,15.8263,1016.51,Partly cloudy throughout the day.


In [5]:
df['formatted_date'] = pd.to_datetime(df['formatted_date'].apply(lambda x: x[:10]))

In [6]:
df = df.sort_values('formatted_date').reset_index(drop=True)

In [7]:
# df['summary'].unique().shape[0] + df['precip_type'].unique().shape[0]

In [8]:
# df.isna().sum()

In [9]:
# counts = np.array([df[df['formatted_date'] == i].shape[0] for i in df['formatted_date'].unique()])

In [10]:
# np.unique(counts)

In [11]:
# np.sort(df['formatted_date'].unique())

In [12]:
new_df = pd.concat([df[df['formatted_date'] == i].sample(12) for i in np.sort(df['formatted_date'].unique())], axis=0).reset_index(drop=True)

In [13]:
y = np.array([new_df['daily_summary'].values[idx:idx+12][0] for idx in range(0, new_df['daily_summary'].shape[0], 12)])

In [14]:
value_counts = pd.DataFrame({'daily_summary':y})
value_counts = value_counts.value_counts('daily_summary').reset_index()

In [15]:
removed_classes = set(value_counts[value_counts[0] < 5]['daily_summary'].values.tolist())

In [16]:
new_df = new_df[~new_df['daily_summary'].isin(removed_classes)]

In [17]:
y = np.array([new_df['daily_summary'].values[idx:idx+12][0] for idx in range(0, new_df['daily_summary'].shape[0], 12)])
X = new_df.drop(columns=['formatted_date', 'summary', 'precip_type', 'daily_summary']).values

le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [18]:
X = X.reshape(-1, 12, 7)

In [19]:
X.shape, y.shape

((3809, 12, 7), (3809,))

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [21]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [22]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


2022-05-21 17:39:18.292189: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-21 17:39:18.366602: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2022-05-21 17:39:18.366844: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 1 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-05-21 17:39:18.367085: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-05-21 17:39:18.368935: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

In [23]:
with strategy.scope():
    model = Sequential()
    model.add(layers.Input((12, 7)))
    model.add(layers.Flatten())
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(128, activation='relu'))
    # model.add(Dropout(0.4))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1, validation_split=0.1)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

2022-05-21 17:39:23.313410: W tensorflow/core/framework/op_kernel.cc:1730] OP_REQUIRES failed at cast_op.cc:123 : Unimplemented: Cast string to float is not supported
2022-05-21 17:39:23.338376: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
